In [ ]:
import numpy as np
import os
from skimage import io, img_as_ubyte
from skimage.transform import resize
import h5py
from skimage.filters import roberts, sobel
from skimage.color import rgb2lab, rgb2gray
import cv2
from skimage.filters.rank import entropy
from skimage.morphology import disk

In [ ]:
def extract_custom_features(img):
    # Color features
    LAB_img = rgb2lab(img)
    A_img = LAB_img[:,:,1]
    A_feat = A_img.mean()

    B_img = LAB_img[:,:,2]
    B_feat = B_img.mean()

    # Textural features based on the gray image
    gray_img = rgb2gray(img)
    gray_img = resize(gray_img, (256,256)) # Resize to smaller size
    gray_img = img_as_ubyte(gray_img)

    # Entropy
    entropy_img = entropy(gray_img, disk(3))
    entropy_mean = entropy_img.mean()
    entropy_std = entropy_img.std()

    roberts_img = roberts(gray_img)
    roberts_mean = roberts_img.mean()

    sobel_img = sobel(gray_img)
    sobel_mean = sobel_img.mean()

    # Gabor filters
    kernel1 = cv2.getGaborKernel((9, 9), 3, np.pi/4, np.pi, 0.5, 0, ktype=cv2.CV_32F)
    gabor1 = (cv2.filter2D(gray_img, cv2.CV_8UC3, kernel1)).mean()

    kernel2 = cv2.getGaborKernel((9, 9), 3, np.pi/2, np.pi/4, 0.9, 0, ktype=cv2.CV_32F)
    gabor2 = (cv2.filter2D(gray_img, cv2.CV_8UC3, kernel2)).mean()

    kernel3 = cv2.getGaborKernel((9, 9), 5, np.pi/2, np.pi/2, 0.1, 0, ktype=cv2.CV_32F)
    gabor3 = (cv2.filter2D(gray_img, cv2.CV_8UC3, kernel3)).mean()

    custom_features = np.array([A_feat, B_feat, entropy_mean, entropy_std, roberts_mean,
                                sobel_mean, gabor1, gabor2, gabor3])

    return custom_features

In [ ]:
def is_image_file(filename):
    valid_extensions = (
        '.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.gif', '.webp',
        '.heif', '.heic', '.ppm', '.pgm', '.pbm', '.pnm', '.svg', '.ico'
    )
    return filename.lower().endswith(valid_extensions)


In [ ]:
path = "all_images/"
feats = []
names = []

for im in os.listdir(path):  # iterate through all images to extract features
    if is_image_file(im):  # Check if it's an image file
        print("Extracting features from image - ", im)
        img = io.imread(os.path.join(path, im))  # Read the image

        # Extract features
        X = extract_custom_features(img)
        feats.append(X)
        names.append(im)
    else:
        print(f"Skipping non-image file: {im}")

# Convert the list of features to a numpy array after the loop
feats = np.array(feats)

# Save the extracted features and names of images into an h5 file
feature_file = "CustomFeatures.h5"
print("Saving features to h5 file")

h5f = h5py.File(feature_file, 'w')
h5f.create_dataset('dataset_1', data=feats)
h5f.create_dataset('dataset_2', data=np.string_(names))
h5f.close()

Extracting features from image -  horse2.jpg
Extracting features from image -  monkey2.jpg
Extracting features from image -  horse1.jpg
Extracting features from image -  2.png
Extracting features from image -  1.png
Extracting features from image -  zebra1.jpg
Extracting features from image -  zebra2.jpg
Extracting features from image -  4.png
Extracting features from image -  tiger1.jpg
Extracting features from image -  tiger2.jpg
Extracting features from image -  monkey1.jpg
Extracting features from image -  chihuahua.JPG
Skipping non-image file: .ipynb_checkpoints
Extracting features from image -  irish_terrier.JPG
Saving features to h5 file


In [ ]:
h5f = h5py.File("CustomFeatures.h5",'r')
feats = h5f['dataset_1'][:]
imgNames = h5f['dataset_2'][:]
h5f.close()

In [ ]:
queryImg = io.imread("query_images/tiger3.jpg")

In [ ]:
X = extract_custom_features(queryImg)

In [ ]:
scores = []
from scipy import spatial
for i in range(feats.shape[0]):
    score = 1-spatial.distance.cosine(X, feats[i])
    scores.append(score)
scores = np.array(scores)
rank_ID = np.argsort(scores)[::-1]
rank_score = scores[rank_ID]

In [ ]:
max_num_matches = 3
imlist = [imgNames[index] for i,index in enumerate(rank_ID[0:max_num_matches])]
print("top %d images in order are: " %max_num_matches, imlist)

top 3 images in order are:  [b'tiger1.jpg', b'monkey1.jpg', b'tiger2.jpg']
